**Connect google drive**

In [1]:
#Enabling NVIDIA GPU is enabled
!nvidia-smi

Thu Dec 15 22:50:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Backup SOP.gdoc'		      Movies
'Colab Notebooks'		     'My Drive'
'Copy of Train_YoloV3 .ipynb'	      Nikhil_Sir_LOR.gdoc
 DCIM				      PDFfiles
'Dhvani Kothari.PDF'		      Photos
'Dhvani_Kothari_Resume (1).pdf'       Pictures
 Dhvani_Kothari_Resume.pdf	      Practitioner_112.PDF
 dk				     'Resume (1).gdoc'
 Dk_backup			     'Resume (2).gdoc'
 Documents			      Resume.gdoc
 Download			      Snapchat
'Final - STATEMENT OF PURPOSE.gdoc'  'STATEMENT OF PURPOSE.gdoc'
'Getting started.pdf'		     'Surabhi Resume.gdoc'
 Goa				     'Surabhi Resume.pdf'
 GRE				      team36.PDF
 HdfcDoc			      Wajgi_Mam_LOR.gdoc
 J-Bday				      yolov3
 JK


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.16 MiB | 18.33 MiB/s, done.
Resolving deltas: 100% (10405/10405), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# changing makefile to have GPU and OPENCV enabled
%cd /content/darknet

!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

In [5]:
!./darknet

usage: ./darknet <function>


**3) Configure Darknet network for training YOLO V3**

In [6]:
!cp cfg/yolov3.cfg cfg/yolov3_new_cfg.cfg

In [7]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_new_cfg.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_new_cfg.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_new_cfg.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_new_cfg.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_new_cfg.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_new_cfg.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_new_cfg.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_new_cfg.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_new_cfg.cfg

In [8]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [9]:
!echo "Firearm" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [10]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-12-15 22:52:06--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  20.7MB/s    in 11s     

2022-12-15 22:52:17 (14.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [11]:
%cd /content/darknet/data/obj/
!pwd
!ls

/content/darknet/data/obj
/content/darknet/data/obj


**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [12]:
#!unzip /mydrive/yolov3/images.zip -d data/obj

In [13]:
#!unzip /mydrive/yolov3/images.zip -d /mydrive/yolov3/images

In [14]:
!cp -r /mydrive/yolov3/images/images/* /content/darknet/data/obj/

In [15]:
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"/content/darknet/data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # getting image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
        if numbers:

            # Defining coordinates
            text = "{} {} {} {} {}".format(0, numbers[0], numbers[1], numbers[2], numbers[3])
            text_converted.append(text)
            print(i, file_path)
            print(text)
        # Writing file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

1

['1']
16 65 148 124

['16', '65', '148', '124']
0 /content/darknet/data/obj/115.txt
0 16 65 148 124
1

['1']
70 78 122 122

['70', '78', '122', '122']
1 /content/darknet/data/obj/305.txt
0 70 78 122 122
2

['2']
112 86 142 163

['112', '86', '142', '163']
164 94 200 154

['164', '94', '200', '154']
2 /content/darknet/data/obj/311.txt
0 164 94 200 154
1

['1']
62 124 140 167

['62', '124', '140', '167']
3 /content/darknet/data/obj/268.txt
0 62 124 140 167
1

['1']
66 53 233 124

['66', '53', '233', '124']
4 /content/darknet/data/obj/321.txt
0 66 53 233 124
2

['2']
74 36 96 56

['74', '36', '96', '56']
213 56 231 83

['213', '56', '231', '83']
5 /content/darknet/data/obj/93.txt
0 213 56 231 83
4

['4']
98 65 264 112

['98', '65', '264', '112']
125 18 311 67

['125', '18', '311', '67']
17 7 109 114

['17', '7', '109', '114']
310 5 397 119

['310', '5', '397', '119']
6 /content/darknet/data/obj/87.txt
0 310 5 397 119
1

['1']
87 70 156 180

['87', '70', '156', '180']
7 /content/darknet

In [16]:
import glob
images_list = glob.glob("/content/darknet/data/obj/*.jpeg")
print(images_list)

['/content/darknet/data/obj/108.jpeg', '/content/darknet/data/obj/130.jpeg', '/content/darknet/data/obj/4.jpeg', '/content/darknet/data/obj/61.jpeg', '/content/darknet/data/obj/245.jpeg', '/content/darknet/data/obj/270.jpeg', '/content/darknet/data/obj/111.jpeg', '/content/darknet/data/obj/317.jpeg', '/content/darknet/data/obj/209.jpeg', '/content/darknet/data/obj/39.jpeg', '/content/darknet/data/obj/193.jpeg', '/content/darknet/data/obj/65.jpeg', '/content/darknet/data/obj/212.jpeg', '/content/darknet/data/obj/121.jpeg', '/content/darknet/data/obj/120.jpeg', '/content/darknet/data/obj/186.jpeg', '/content/darknet/data/obj/249.jpeg', '/content/darknet/data/obj/152.jpeg', '/content/darknet/data/obj/154.jpeg', '/content/darknet/data/obj/14.jpeg', '/content/darknet/data/obj/1.jpeg', '/content/darknet/data/obj/103.jpeg', '/content/darknet/data/obj/278.jpeg', '/content/darknet/data/obj/216.jpeg', '/content/darknet/data/obj/151.jpeg', '/content/darknet/data/obj/302.jpeg', '/content/darknet/d

In [17]:
#Creating training.txt file
file = open("/content/darknet/data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [18]:
%cd /content/darknet

/content/darknet


**6) Start the training**

In [ ]:
# Starting the training
!./darknet detector train /content/darknet/data/obj.data /content/darknet/cfg/yolov3_training.cfg /content/darknet/darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 234.379089, iou_loss = 0.000000, total_loss = 234.379089 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 541.503113, iou_loss = 0.000000, total_loss = 541.503113 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 1896.146606, iou_loss = 0.000000, total_loss = 1896.146606 

 Wrong annotation: x = 0, y = 0, < 0 or > 1, file: /content/darknet/data/obj/182.txt 
 total_bbox = 1779, rewritten_bbox = 0.000000 % 

 Wrong annotation: x = 0, y = 0, < 0 or > 1, file: /content/darknet/data/obj/184.txt 

 Wrong annotation: x = 0, y = 0, < 0 or > 1, file: /content/darknet/data/obj/10.txt 

 Wrong annotation: x = 0, y = 0, < 0 or > 1, file: /content/darknet/data/obj/249.txt 

 Wrong annotation: x = 0, y = 0, 